# Kolecto Churn Prediction Analysis

This notebook analyzes user behavior during the 15-day trial period to predict conversion to paid subscriptions.
We use Logistic Regression, XGBoost, and LightGBM models.

## Enhancements
- **Configuration**: Hyperparameters are managed using Pydantic models.
- **Visualization**: Added EDA plots for target distribution, correlations, and feature distributions.
- **Explanations**: Detailed reasoning for preprocessing steps.

In [ ]:
# Imports
import pandas as pd
import numpy as np
from pydantic import BaseModel, Field
from typing import Optional, List, Dict, Any
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report, confusion_matrix, precision_recall_curve, auc, brier_score_loss
from sklearn.calibration import calibration_curve
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')

## 1. Configuration with Pydantic

We use Pydantic to strictly define our configuration, including data splitting parameters and model hyperparameters. This makes the experiment reproducible and easy to tune.

In [ ]:
class DataConfig(BaseModel):
    subscriptions_path: str = Field('Data/subscriptions.csv', description="Path to subscriptions data")
    daily_usage_path: str = Field('Data/daily_usage.csv', description="Path to daily usage data")
    test_size: float = Field(0.2, description="Test set size")
    random_state: int = Field(42, description="Random seed for reproducibility")

class LogisticRegressionConfig(BaseModel):
    max_iter: int = Field(1000, description="Maximum number of iterations")
    C: float = Field(1.0, description="Inverse of regularization strength")
    class_weight: str = Field('balanced', description="Handle class imbalance")

class XGBoostConfig(BaseModel):
    n_estimators: int = Field(500, description="Number of boosting rounds")
    max_depth: int = Field(7, description="Maximum tree depth")
    learning_rate: float = Field(0.05, description="Learning rate")
    subsample: float = Field(0.8, description="Subsample ratio of training instances")
    colsample_bytree: float = Field(0.8, description="Subsample ratio of columns when constructing each tree")
    min_child_weight: int = Field(20, description="Minimum sum of instance weight needed in a child")
    early_stopping_rounds: int = Field(50, description="Early stopping rounds")
    random_state: int = Field(42, description="Random seed")

class LightGBMConfig(BaseModel):
    n_estimators: int = Field(500, description="Number of boosting rounds")
    num_leaves: int = Field(31, description="Maximum number of leaves in one tree")
    max_depth: int = Field(-1, description="Maximum tree depth, -1 means no limit")
    learning_rate: float = Field(0.05, description="Learning rate")
    feature_fraction: float = Field(0.8, description="Fraction of features for each tree")
    bagging_fraction: float = Field(0.8, description="Fraction of data for bagging")
    bagging_freq: int = Field(5, description="Frequency for bagging")
    min_data_in_leaf: int = Field(20, description="Minimum number of data in one leaf")
    early_stopping_rounds: int = Field(50, description="Early stopping rounds")
    random_state: int = Field(42, description="Random seed")

class ExperimentConfig(BaseModel):
    data: DataConfig = DataConfig()
    lr: LogisticRegressionConfig = LogisticRegressionConfig()
    xgb: XGBoostConfig = XGBoostConfig()
    lgb: LightGBMConfig = LightGBMConfig()

# Instantiate Configuration
config = ExperimentConfig()
print("Current Configuration:")
print(config.model_dump_json(indent=2))

## 2. Data Loading

In [ ]:
# Load Data
subscriptions_df = pd.read_csv(config.data.subscriptions_path)
daily_usage_df = pd.read_csv(config.data.daily_usage_path)

print(f'Loaded {len(subscriptions_df)} subscriptions and {len(daily_usage_df)} daily usage records')
print(f'Subscriptions shape: {subscriptions_df.shape}')
print(f'Daily Usage shape: {daily_usage_df.shape}')

### 2.2 Data Quality & Cleaning
Remove duplicates, handle missing values strategically, and detect anomalies.

In [ ]:
# Remove duplicates
print(f'Subscriptions before deduplication: {len(subscriptions_df)}')
subscriptions_df = subscriptions_df.drop_duplicates(subset=['subscription_id'], keep='first')
print(f'Subscriptions after deduplication: {len(subscriptions_df)}')

print(f'\nDaily usage before deduplication: {len(daily_usage_df)}')
daily_usage_df = daily_usage_df.drop_duplicates(subset=['subscription_id', 'day_date'], keep='first')
print(f'Daily usage after deduplication: {len(daily_usage_df)}')

### 2.3 Data Preprocessing

In [ ]:
# Parse Dates
date_columns = ['day_date', 'subscription_created_at', 'trial_starts_at', 'trial_ends_at', 'canceled_at', 'first_paid_invoice_paid_at']
for col in date_columns:
    if col in subscriptions_df.columns:
        subscriptions_df[col] = pd.to_datetime(subscriptions_df[col], errors='coerce')
    if col in daily_usage_df.columns:
        daily_usage_df[col] = pd.to_datetime(daily_usage_df[col], errors='coerce')

# Convert revenue range to numeric (midpoint) with strategic imputation
def parse_revenue_range(range_str):
    if pd.isna(range_str) or range_str == 'Unknown':
        return np.nan
    if '<100k€' in str(range_str):
        return 50
    elif '100k€-500k€' in str(range_str):
        return 300
    elif '500k€-1M€' in str(range_str):
        return 750
    elif '1M€-5M€' in str(range_str):
        return 3000
    elif '5M€-10M€' in str(range_str):
        return 7500
    elif '>10M€' in str(range_str):
        return 15000
    return np.nan

subscriptions_df['revenue_mid'] = subscriptions_df['revenue_range'].apply(parse_revenue_range)

# Strategic imputation with median for revenue
revenue_median = subscriptions_df['revenue_mid'].median()
subscriptions_df['revenue_mid'] = subscriptions_df['revenue_mid'].fillna(revenue_median)
print(f'Imputed missing revenue with median: {revenue_median:.0f}k€')

# Convert employee count range
def parse_employee_range(range_str):
    if pd.isna(range_str) or range_str == 'Unknown':
        return np.nan
    if '0-10' in str(range_str):
        return 5
    elif '11-50' in str(range_str):
        return 30
    elif '51-250' in str(range_str):
        return 150
    elif '>250' in str(range_str):
        return 500
    return np.nan

subscriptions_df['employee_mid'] = subscriptions_df['employee_count'].apply(parse_employee_range)

# Strategic imputation with median for employees
employee_median = subscriptions_df['employee_mid'].median()
subscriptions_df['employee_mid'] = subscriptions_df['employee_mid'].fillna(employee_median)
print(f'Imputed missing employee count with median: {employee_median:.0f}')

# Normalize ranges (min-max scaling)
from sklearn.preprocessing import MinMaxScaler
scaler_revenue = MinMaxScaler()
scaler_employee = MinMaxScaler()

subscriptions_df['revenue_mid_normalized'] = scaler_revenue.fit_transform(subscriptions_df[['revenue_mid']])
subscriptions_df['employee_mid_normalized'] = scaler_employee.fit_transform(subscriptions_df[['employee_mid']])
print('\nNormalized revenue and employee ranges to [0,1]')

# Company age
subscriptions_df['company_age_in_years'] = pd.to_numeric(subscriptions_df['company_age_in_years'], errors='coerce')
age_median = subscriptions_df['company_age_in_years'].median()
subscriptions_df['company_age_in_years'] = subscriptions_df['company_age_in_years'].fillna(age_median)
print(f'Imputed missing company age with median: {age_median:.0f} years')

# Filter for 15-day trials
subscriptions_df['trial_duration'] = (subscriptions_df['trial_ends_at'] - subscriptions_df['trial_starts_at']).dt.days
subscriptions_df = subscriptions_df[subscriptions_df['trial_duration'] == 15].copy()

# Define Target: Converted = 1 if first_paid_invoice_paid_at is present
subscriptions_df['converted'] = subscriptions_df['first_paid_invoice_paid_at'].notna().astype(int)

### 3.2 Aggregating Usage Data
We sum the daily usage logs for each subscription to get a total usage profile for the trial period.

### 3.3 Anomaly Detection
We identify and handle potential outliers in the usage data that could skew our models. This involves using Isolation Forest to detect anomalies in aggregated usage features.

In [ ]:
from sklearn.ensemble import IsolationForest

# Identify numerical columns for anomaly detection (excluding subscription_id and day_date)
usage_numerical_cols = [col for col in daily_usage_df.columns if daily_usage_df[col].dtype in ['int64', 'float64'] and col not in ['subscription_id', 'day_date', 'day_number']]

if not usage_numerical_cols:
    print("No numerical usage columns found for anomaly detection.")
else:
    # Impute NaNs before anomaly detection if any exist
    # For simplicity, using median imputation here, but more sophisticated methods could be used.
    for col in usage_numerical_cols:
        if daily_usage_df[col].isnull().any():
            median_val = daily_usage_df[col].median()
            daily_usage_df[col] = daily_usage_df[col].fillna(median_val)

    # Apply Isolation Forest to detect anomalies in the usage data
    # We'll fit it on the raw daily usage data, then use it to flag subscriptions
    iso_forest = IsolationForest(random_state=config.data.random_state, contamination=0.01) # Assuming 1% anomalies
    daily_usage_df['anomaly_score'] = iso_forest.fit_predict(daily_usage_df[usage_numerical_cols])

    # Aggregate anomaly scores to the subscription level (e.g., if any day is an anomaly, flag the subscription)
    subscription_anomalies = daily_usage_df.groupby('subscription_id')['anomaly_score'].apply(lambda x: (x == -1).any()).reset_index()
    subscription_anomalies.columns = ['subscription_id', 'is_usage_anomaly']

    # Merge this back to the main subscriptions_df or df later if needed
    # For now, we'll just print the count of anomalous subscriptions
    num_anomalous_subscriptions = subscription_anomalies['is_usage_anomaly'].sum()
    print(f"Detected {num_anomalous_subscriptions} subscriptions with anomalous usage patterns.")

    # Optionally, remove or treat anomalous subscriptions
    # For this notebook, we'll keep them but acknowledge their presence.
    # If we were to remove them:
    # subscriptions_df = subscriptions_df.merge(subscription_anomalies, on='subscription_id', how='left')
    # subscriptions_df = subscriptions_df[~subscriptions_df['is_usage_anomaly']].drop(columns=['is_usage_anomaly'])
    # print(f"Remaining subscriptions after removing anomalies: {len(subscriptions_df)}")

In [ ]:
usage_cols = [col for col in daily_usage_df.columns if col not in ['subscription_id', 'day_date']]

# Convert day_date to datetime
daily_usage_df['day_date'] = pd.to_datetime(daily_usage_df['day_date'])

# Calculate trial start date for each subscription
trial_start_dates = subscriptions_df[['subscription_id', 'trial_starts_at']].copy()
daily_usage_df = daily_usage_df.merge(trial_start_dates, on='subscription_id', how='left')
daily_usage_df['day_number'] = (daily_usage_df['day_date'] - daily_usage_df['trial_starts_at']).dt.days

# Aggregate total usage (as before)
usage_agg = daily_usage_df.groupby('subscription_id')[usage_cols].sum().reset_index()
usage_agg.columns = ['subscription_id'] + [f'total_{col}' for col in usage_cols]

# Early trial activity (first 3 days: days 0-2)
early_usage = daily_usage_df[daily_usage_df['day_number'].between(0, 2)].groupby('subscription_id')[usage_cols].sum().reset_index()
early_usage.columns = ['subscription_id'] + [f'early_{col}' for col in usage_cols]

# Late trial activity (last 3 days: days 12-14)
late_usage = daily_usage_df[daily_usage_df['day_number'].between(12, 14)].groupby('subscription_id')[usage_cols].sum().reset_index()
late_usage.columns = ['subscription_id'] + [f'late_{col}' for col in usage_cols]

# Calculate total daily activity for trend analysis
daily_usage_df['daily_total_activity'] = daily_usage_df[usage_cols].sum(axis=1)

# Engagement trend (slope of activity over time)
def calculate_trend(group):
    if len(group) < 2:
        return 0
    x = group['day_number'].values
    y = group['daily_total_activity'].values
    if np.std(x) == 0:
        return 0
    return np.corrcoef(x, y)[0, 1] if len(x) > 1 else 0

engagement_trend = daily_usage_df.groupby('subscription_id').apply(calculate_trend).reset_index()
engagement_trend.columns = ['subscription_id', 'engagement_trend']

# Activity variance and peak day
activity_stats = daily_usage_df.groupby('subscription_id').agg({
    'daily_total_activity': ['std', 'max']
}).reset_index()
activity_stats.columns = ['subscription_id', 'activity_std', 'activity_max']
activity_stats['activity_std'] = activity_stats['activity_std'].fillna(0)

# Calculate peak activity day separately
peak_days = daily_usage_df.loc[daily_usage_df.groupby('subscription_id')['daily_total_activity'].idxmax()][['subscription_id', 'day_number']]
peak_days.columns = ['subscription_id', 'peak_activity_day']
activity_stats = activity_stats.merge(peak_days, on='subscription_id', how='left')
activity_stats['peak_activity_day'] = activity_stats['peak_activity_day'].fillna(0)

# Feature diversity (number of distinct features used)
feature_diversity = daily_usage_df.groupby('subscription_id')[usage_cols].apply(lambda x: (x > 0).any().sum()).reset_index()
feature_diversity.columns = ['subscription_id', 'feature_diversity']

# Merge all features
df = pd.merge(subscriptions_df, usage_agg, on='subscription_id', how='left')
df = pd.merge(df, early_usage, on='subscription_id', how='left')
df = pd.merge(df, late_usage, on='subscription_id', how='left')
df = pd.merge(df, engagement_trend, on='subscription_id', how='left')
df = pd.merge(df, activity_stats, on='subscription_id', how='left')
df = pd.merge(df, feature_diversity, on='subscription_id', how='left')

# Fill NaN values
new_feature_cols = [col for col in df.columns if any(prefix in col for prefix in ['total_', 'early_', 'late_', 'engagement', 'activity', 'feature_diversity'])]
df[new_feature_cols] = df[new_feature_cols].fillna(0)

print(f'Dataset shape after basic feature engineering: {df.shape}')

### 3.4 Advanced Temporal Features ⭐ HIGH IMPACT
Sophisticated time-based patterns: milestones, velocity, first-action timing.

In [ ]:
# A. Cumulative Engagement by Milestone Days
for milestone in [3, 7, 10, 12]:
    milestone_data = daily_usage_df[daily_usage_df['day_number'] <= milestone]
    cumulative = milestone_data.groupby('subscription_id')['daily_total_activity'].sum().reset_index()
    cumulative.columns = ['subscription_id', f'cumulative_day{milestone}']
    df = df.merge(cumulative, on='subscription_id', how='left')

print('Added cumulative milestone features')

# B. Time-to-First-Action (top 10 features)
for col in usage_cols[:10]:
    first_use = daily_usage_df[daily_usage_df[col] > 0].groupby('subscription_id')['day_number'].min().reset_index()
    first_use.columns = ['subscription_id', f'first_{col}']
    df = df.merge(first_use, on='subscription_id', how='left')
    df[f'first_{col}'] = df[f'first_{col}'].fillna(16)  # Never used = day 16
    df[f'never_{col}'] = (df[f'first_{col}'] == 16).astype(int)

print('Added time-to-first-action features')

# C. Activity Velocity
velocity = daily_usage_df.groupby('subscription_id').apply(
    lambda x: x['daily_total_activity'].iloc[-3:].mean() - x['daily_total_activity'].iloc[:3].mean() if len(x) >= 6 else 0
).reset_index()
velocity.columns = ['subscription_id', 'activity_velocity']
df = df.merge(velocity, on='subscription_id', how='left')

# D. Active Days Ratio
active_ratio = daily_usage_df.groupby('subscription_id').apply(
    lambda x: (x['daily_total_activity'] > 0).mean()
).reset_index()
active_ratio.columns = ['subscription_id', 'active_days_ratio']
df = df.merge(active_ratio, on='subscription_id', how='left')

# E. Weekend Activity
daily_usage_df['is_weekend'] = daily_usage_df['day_date'].dt.dayofweek >= 5
weekend = daily_usage_df.groupby('subscription_id').apply(
    lambda x: x[x['is_weekend']]['daily_total_activity'].sum() / (x['daily_total_activity'].sum() + 1)
).reset_index()
weekend.columns = ['subscription_id', 'weekend_ratio']
df = df.merge(weekend, on='subscription_id', how='left')

print(f'Final shape with advanced temporal features: {df.shape}')

## 4. Exploratory Data Analysis (EDA)

### 4.1 Target Distribution
Checking the balance of our target variable is crucial. Imbalanced datasets might require techniques like SMOTE or class weighting.

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x='converted', data=df, palette='viridis')
plt.title('Distribution of Target Variable (Converted)')
plt.xlabel('Converted (0=No, 1=Yes)')
plt.ylabel('Count')
plt.show()

### 4.2 Feature Correlations
We analyze the correlation between numerical features and the target to identify strong predictors.

In [ ]:
# Select numerical columns for correlation
num_feature_cols = [col for col in df.columns if any(prefix in col for prefix in ['total_', 'early_', 'late_', 'engagement', 'activity', 'feature_diversity'])]
num_cols_for_corr = ['company_age_in_years', 'converted'] + num_feature_cols[:10] # limiting for readability
corr_matrix = df[num_cols_for_corr].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix (Top Features)')
plt.tight_layout()
plt.show()

## 5. Pipeline & Modeling

### 5.1 Preprocessing Pipeline
- **Numerical Features**: Imputed with Median (robust to outliers), Scaled (StandardScaler) for Logistic Regression.
- **Categorical Features**: Imputed with 'Unknown', One-Hot Encoded to handle categorical data for all models.

In [ ]:
categorical_features = ['vendor', 'v2_segment', 'naf_section', 'revenue_range', 'employee_count', 'regional_pole', 'legal_structure', 'company_age_group']
numerical_features = ['company_age_in_years'] + [col for col in df.columns if any(prefix in col for prefix in ['total_', 'early_', 'late_', 'engagement', 'activity', 'feature_diversity'])]

# Clean Numerical for split
df['company_age_in_years'] = pd.to_numeric(df['company_age_in_years'], errors='coerce')

X = df[categorical_features + numerical_features]
y = df['converted']

# Split using Config
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=config.data.test_size, random_state=config.data.random_state
)

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

### 5.2 Train/Validation Split
We create a validation set from the training data for early stopping. This helps prevent overfitting by stopping training when validation performance plateaus.

In [ ]:
# Split training data further for validation (for early stopping)
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=config.data.random_state, stratify=y_train
)

print(f'Training set: {X_train_split.shape[0]} samples')
print(f'Validation set: {X_val.shape[0]} samples')
print(f'Test set: {X_test.shape[0]} samples')

### 5.3 Model Training with Regularization
We train three models with advanced regularization techniques:
- **Feature/Bagging sampling**: Reduces overfitting through randomization
- **Early stopping**: Finds optimal number of iterations
- **Leaf constraints**: Prevents overly specific splits

In [ ]:
results = {}

# 1. Logistic Regression
lr_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(max_iter=config.lr.max_iter, C=config.lr.C, class_weight=config.lr.class_weight))])
lr_model.fit(X_train, y_train)
results['Logistic Regression'] = lr_model.predict_proba(X_test)[:, 1]

# 2. XGBoost with Advanced Regularization
try:
    import xgboost as xgb
    # Calculate scale_pos_weight for class imbalance
    scale_pos_weight = (y_train_split == 0).sum() / (y_train_split == 1).sum()
    
    # Preprocess data for early stopping
    preprocessor_xgb = preprocessor.fit(X_train_split, y_train_split)
    X_train_xgb = preprocessor_xgb.transform(X_train_split)
    X_val_xgb = preprocessor_xgb.transform(X_val)
    
    xgb_clf = xgb.XGBClassifier(
        n_estimators=config.xgb.n_estimators,
        max_depth=config.xgb.max_depth,
        learning_rate=config.xgb.learning_rate,
        subsample=config.xgb.subsample,
        colsample_bytree=config.xgb.colsample_bytree,
        min_child_weight=config.xgb.min_child_weight,
        random_state=config.xgb.random_state,
        scale_pos_weight=scale_pos_weight,
        use_label_encoder=False,
        eval_metric='logloss'
    )
    from xgboost.callback import EarlyStopping
    xgb_clf.fit(
        X_train_xgb, y_train_split,
        eval_set=[(X_val_xgb, y_val)],
        callbacks=[EarlyStopping(rounds=config.xgb.early_stopping_rounds, save_best=True)],
        verbose=False
    )
    print(f'XGBoost stopped at iteration: {xgb_clf.best_iteration + 1}/{config.xgb.n_estimators}')
    results['XGBoost'] = xgb_clf.predict_proba(preprocessor_xgb.transform(X_test))[:, 1]
except ImportError:
    print("XGBoost not installed.")

# 3. LightGBM with Advanced Regularization
try:
    import lightgbm as lgb
    
    # Preprocess data for early stopping
    preprocessor_lgb = preprocessor.fit(X_train_split, y_train_split)
    X_train_lgb = preprocessor_lgb.transform(X_train_split)
    X_val_lgb = preprocessor_lgb.transform(X_val)
    
    lgb_clf = lgb.LGBMClassifier(
        n_estimators=config.lgb.n_estimators,
        num_leaves=config.lgb.num_leaves,
        max_depth=config.lgb.max_depth,
        learning_rate=config.lgb.learning_rate,
        feature_fraction=config.lgb.feature_fraction,
        bagging_fraction=config.lgb.bagging_fraction,
        bagging_freq=config.lgb.bagging_freq,
        min_data_in_leaf=config.lgb.min_data_in_leaf,
        random_state=config.lgb.random_state,
        class_weight='balanced',
        verbose=-1
    )
    lgb_clf.fit(
        X_train_lgb, y_train_split,
        eval_set=[(X_val_lgb, y_val)],
        eval_metric='auc',
        callbacks=[lgb.early_stopping(stopping_rounds=config.lgb.early_stopping_rounds, verbose=False)]
    )
    print(f'LightGBM stopped at iteration: {lgb_clf.best_iteration_}/{config.lgb.n_estimators}')
    results['LightGBM'] = lgb_clf.predict_proba(preprocessor_lgb.transform(X_test))[:, 1]
except ImportError:
    print("LightGBM not installed.")

## 6. Evaluation
We compare the models using multiple metrics and cross-validation for robustness.

In [ ]:
# Test set performance
print('Test Set Results:')
print('-' * 60)
model_metrics = {}
for name, y_prob in results.items():
    y_pred = (y_prob >= 0.5).astype(int)
    acc = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_prob)
    print(f'{name}: Accuracy={acc:.4f}, AUC={auc_score:.4f}')
    model_metrics[name] = {'Accuracy': acc, 'AUC': auc_score}
print()


### 6.1 Model Accuracy Comparison
Visual comparison of model performance across key metrics.

In [ ]:
# Create accuracy comparison bar plot
import matplotlib.pyplot as plt
import numpy as np

models = list(model_metrics.keys())
accuracies = [model_metrics[m]['Accuracy'] for m in models]
aucs = [model_metrics[m]['AUC'] for m in models]

x = np.arange(len(models))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, accuracies, width, label='Accuracy', alpha=0.8, color='steelblue')
bars2 = ax.bar(x + width/2, aucs, width, label='AUC', alpha=0.8, color='coral')

ax.set_ylabel('Score', fontsize=12)
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print('\nBest Model by Accuracy:', max(model_metrics.items(), key=lambda x: x[1]['Accuracy'])[0])
print('Best Model by AUC:', max(model_metrics.items(), key=lambda x: x[1]['AUC'])[0])

### 6.2 ROC Curves

In [ ]:
plt.figure(figsize=(10, 8))
for name, y_prob in results.items():
    auc = roc_auc_score(y_test, y_prob)
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label=f"{name} (AUC = {auc:.3f})", linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curves Comparison', fontsize=14)
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()

### 6.2 Precision-Recall Curves
Important for imbalanced datasets where we care about positive class prediction quality.

In [ ]:
plt.figure(figsize=(10, 8))
for name, y_prob in results.items():
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    plt.plot(recall, precision, label=f"{name}", linewidth=2)

plt.xlabel('Recall', fontsize=12)
plt.ylabel('Precision', fontsize=12)
plt.title('Precision-Recall Curves', fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.show()

### 6.3 Confusion Matrices

In [ ]:
fig, axes = plt.subplots(1, len(results), figsize=(5*len(results), 4))
if len(results) == 1:
    axes = [axes]

for idx, (name, y_prob) in enumerate(results.items()):
    y_pred = (y_prob >= 0.5).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(f'{name}')
    axes[idx].set_xlabel('Predicted')
    axes[idx].set_ylabel('Actual')
    axes[idx].set_xticklabels(['Not Converted', 'Converted'])
    axes[idx].set_yticklabels(['Not Converted', 'Converted'])

plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance (LightGBM)
if 'LightGBM' in results:
    feature_names = numerical_features + list(preprocessor_lgb.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out())
    importances = lgb_clf.feature_importances_
    indices = np.argsort(importances)[::-1][:20]
    
    plt.figure(figsize=(10, 8))
    plt.title("Top 20 Feature Importances (LightGBM)", fontsize=14)
    plt.barh(range(len(indices)), importances[indices], align="center")
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Importance', fontsize=12)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 7. Model Interpretability with SHAP
SHAP (SHapley Additive exPlanations) helps us understand which features drive predictions and by how much.

In [ ]:
import shap

# Use best performing model (typically XGBoost or LightGBM)
best_model_name = max(results.items(), key=lambda x: roc_auc_score(y_test, x[1]))[0]
print(f'Generating SHAP explanations for: {best_model_name}')

if best_model_name == 'XGBoost':
    explainer = shap.TreeExplainer(xgb_clf)
    shap_values = explainer(preprocessor_xgb.transform(X_test))
    feature_names_shap = numerical_features + list(preprocessor_xgb.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out())
elif best_model_name == 'LightGBM':
    explainer = shap.TreeExplainer(lgb_clf)
    shap_values = explainer(preprocessor_lgb.transform(X_test))
    feature_names_shap = numerical_features + list(preprocessor_lgb.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out())

### 7.1 SHAP Summary Plot
Shows the most important features and their impact on predictions.

In [ ]:
if best_model_name in ['XGBoost', 'LightGBM']:
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values.values, features=shap_values.data, feature_names=feature_names_shap, max_display=20, show=False)
    plt.tight_layout()
    plt.show()

### 7.2 Top Feature Drivers
Quantifying which features matter most for conversion predictions.

In [ ]:
if best_model_name in ['XGBoost', 'LightGBM']:
    # Calculate mean absolute SHAP values
    mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
    feature_importance_df = pd.DataFrame({
        'feature': feature_names_shap,
        'importance': mean_abs_shap
    }).sort_values('importance', ascending=False).head(15)
    
    print('\nTop 15 Features by SHAP Importance:')
    print('=' * 60)
    for idx, row in feature_importance_df.iterrows():
        print(f"{row['feature']:45s}: {row['importance']:.4f}")
    
    # Bar plot
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(feature_importance_df)), feature_importance_df['importance'])
    plt.yticks(range(len(feature_importance_df)), feature_importance_df['feature'])
    plt.xlabel('Mean |SHAP Value|', fontsize=12)
    plt.title('Top 15 Feature Importance (SHAP)', fontsize=14)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

### 7.3 Sample Predictions
Understanding individual predictions with waterfall plots.

In [ ]:
if best_model_name in ['XGBoost', 'LightGBM']:
    # Find a converted and non-converted example
    converted_idx = np.where(y_test == 1)[0][0]
    not_converted_idx = np.where(y_test == 0)[0][0]
    
    print(f'Explaining prediction for converted user (index {converted_idx}):')
    shap.plots.waterfall(shap_values[converted_idx], max_display=15, show=False)
    plt.tight_layout()
    plt.show()
    
    print(f'\nExplaining prediction for non-converted user (index {not_converted_idx}):')
    shap.plots.waterfall(shap_values[not_converted_idx], max_display=15, show=False)
    plt.tight_layout()
    plt.show()

## 8. Calibration Analysis
Calibration measures if predicted probabilities match actual conversion rates. Well-calibrated models are trustworthy for decision-making.

In [ ]:
plt.figure(figsize=(10, 8))

for name, y_prob in results.items():
    fraction_of_positives, mean_predicted_value = calibration_curve(y_test, y_prob, n_bins=10)
    brier = brier_score_loss(y_test, y_prob)
    plt.plot(mean_predicted_value, fraction_of_positives, 's-', label=f'{name} (Brier={brier:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label=' Perfect Calibration')
plt.xlabel('Mean Predicted Probability', fontsize=12)
plt.ylabel('Fraction of Positives', fontsize=12)
plt.title('Calibration Plots\n(Lower Brier Score = Better Calibration)', fontsize=14)
plt.legend(loc='upper left')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print('\nCalibration Scores (Brier Loss - lower is better):')
print('=' * 60)
for name, y_prob in results.items():
    brier = brier_score_loss(y_test, y_prob)
    print(f'{name}: {brier:.4f}')

## 9. Comprehensive Metrics Summary
Final performance summary with all key metrics.

In [ ]:
print('\n' + '=' * 80)
print('FINAL MODEL PERFORMANCE SUMMARY')
print('=' * 80)

for name, y_prob in results.items():
    y_pred = (y_prob >= 0.5).astype(int)
    
    # Calculate metrics
    acc = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    precision, recall, _ = precision_recall_curve(y_test, y_prob)
    pr_auc = auc(recall, precision)
    brier = brier_score_loss(y_test, y_prob)
    
    print(f'\n{name}:')
    print(f'  ROC-AUC:     {roc_auc:.4f}')
    print(f'  PR-AUC:      {pr_auc:.4f}')
    print(f'  Accuracy:    {acc:.4f}')
    print(f'  Brier Score: {brier:.4f}')

print('\n' + '=' * 80)